In [1]:
import warnings
warnings.filterwarnings("ignore")
import os
import numpy as np
import pandas as pd


import matplotlib.pyplot as plt
import seaborn as sns
   
import librosa as lr

import librosa
import speech_recognition as sr

import IPython.display as ipd
import librosa.display

## <font color='#002B54'> Taking Input from  Microphone</font>

In [5]:
import librosa
import speech_recognition as sr

# obtain audio from the microphone
r = sr.Recognizer()
with sr.Microphone() as source:
    print("Hiii SARK's Say something!")
    audio = r.listen(source,timeout=1,phrase_time_limit=4)
print("Recording Done")    
# write audio to a WAV file
with open("output1.wav", "wb") as f:
    f.write(audio.get_wav_data())
    
#Audio to text

txt=sr.AudioFile("output1.wav")

with txt as source:
    audio = r.record(source)
try:
    s = r.recognize_google(audio)
    print("You Said  : "+s)
except Exception as e:
    print("Exception: "+str(e))

ipd.display(ipd.Audio('output1.wav'))

Hiii SARK's Say something!
Recording Done
Exception: 


In [6]:
#Feature Extraction of Audio Files Function 
#Extract features (mfcc, chroma, mel) from a sound file

x=[]
y=[]
def extract_feature(data,sampling_rate):
    result=np.array([])
    #Path=i
    #result=np.hstack((result, Path))
    
    stft = np.abs(librosa.stft(data))
    chromagram = np.mean(librosa.feature.chroma_stft(S=stft, sr=sampling_rate).T, axis=0)
    result=np.hstack((result, chromagram))
   
     
        
    mfcc = np.mean(librosa.feature.mfcc(y=data, sr=sampling_rate, n_mfcc=40).T, axis=0)
    result=np.hstack((result, mfcc))
    

    mel=np.mean(librosa.feature.melspectrogram(data, sr=sampling_rate).T,axis=0)
    result=np.hstack((result, mel))
    

    return result

In [11]:
x=[]

#for i in range (0,20,1):
data, sampling_rate = librosa.load('output1.wav')
feature=extract_feature(data,sampling_rate)
x.append(feature)


In [12]:
import pickle
Pkl_Filename = "new_mlp_74.pkl"
# Loading the Model back from file
with open(Pkl_Filename, 'rb') as file:  
    model= pickle.load(file)

model

MLPClassifier(alpha=0.01, batch_size=16, hidden_layer_sizes=1000,
              learning_rate='adaptive', max_iter=500)

In [14]:
model.predict(x)

array(['sad'], dtype='<U8')

In [30]:
from keras.models import model_from_json
json_file = open('Final_Emotion_Detection_by_Sequential.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)

# load weights into new model
loaded_model.load_weights("saved_models/Final_Emotion_Detection_by_Sequential.h5")
print("Loaded model from disk")
 
# # evaluate loaded model on test data
# loaded_model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])
# score = loaded_model.evaluate(x_test, y_test, verbose=0)
# print("%s: %.2f%%" % (loaded_model.metrics_names[1], score[1]*100))

Loaded model from disk


In [31]:
op=np.expand_dims(y,-1)
preds=loaded_model.predict(op)

In [32]:
predict=pd.DataFrame(preds.argmax(axis=1), columns=['Predicted_y'])

In [33]:
predict["Predicted_y"].map({0:'neutral', 1:'calm', 2:'happy', 3:'sad', 4:'angry', 5:'fear', 6:'disgust', 7:'surprise'})

0    neutral
Name: Predicted_y, dtype: object